# Pipeline 

In [1]:
%load_ext autoreload
%autoreload 2

from averitec import Datapoint
from evidence_generation import GptEvidenceGenerator, GptBatchedEvidenceGenerator, DynamicFewShotBatchedEvidenceGenerator
from classification import DefaultClassifier, HuggingfaceClassifier, AverageEnsembleClassifier, LogRegEnsembleClassifier
from retrieval import SimpleFaissRetriever, Retriever, MmrFaissRetriever, SubqueryRetriever
from pipeline import Pipeline, MockPipeline
import pickle
from labels import label2id, id2label
import numpy as np
from sklearn.metrics import classification_report
import random
from tqdm import tqdm
import htmldate
random.seed(111)

import json

## Pipeline test

In [2]:
split = "test"
path = "/mnt/data/factcheck/averimatec/"
with open(path + f"{split}.json") as f:
    dataset = json.load(f)
    for i in range(len(dataset)):
        dataset[i]["claim_id"] = i
        dataset[i]["split"] = split
    datapoints = [Datapoint.from_dict(d) for d in dataset]

In [3]:
datapoint = Datapoint.from_dict(dataset[150])
datapoint

Datapoint(claim='A picture shows the dead body of Pakistani teenager Arsalan Naseer’s father, who, according to reports, died due to police brutality in August 2023.', claim_id=150, claim_date='2023-08-13', claim_images=['678e463de2f02e5f49857419#CLAIM#0.jpg'], speaker='@AamnaFasihi', original_claim_url='https://web.archive.org/web/20230814193338/https://twitter.com/AamnaFasihi/status/1691137083065118721', reporting_source='Twitter ', location_ISO_code='PK', label='', metadata={'transcription': '', 'media_source': '', 'modality': 'Image-text', 'image_used': 'Yes'}, justification='', split='test')

In [4]:
print(datapoint.speaker)

@AamnaFasihi


In [5]:
# retriever = SimpleFaissRetriever(path="/mnt/data/factcheck/averitec-data/data_store/vecstore/dev/6k")
retriever = MmrFaissRetriever(path=f"/mnt/data/factcheck/averimatec/vector_store/{split}/text/2k_mxbai",k=7, fetch_k=24)
# retriever = MmrFaissRetriever(path=f"/mnt/data/factcheck/averitec-data/data_store/vecstore/{split}/2k")
retrieval_result = retriever(datapoint)
retrieval_result

RetrievalResult(documents=[Document(id='87f8c597-4def-45bb-8452-a632c15b019b', metadata={'url': 'https://www.hrw.org/report/2016/09/27/crooked-system/police-abuse-and-reform-pakistan', 'context_before': 'A few days later, I was in my house preparing to go to sleep when I heard violent knocking on the door. I went out and found two local police constables outside, armed with automatic assault rifles, asking for Shahbaz. I called Shahbaz from his room. They asked him to step outside the house. In the meantime, the local SHO arrived with four more constables in his official vehicle. Upon his arrival, the constables assaulted Shahbaz, kicking and punching him. One of the constables shot him in the right shoulder with a pistol. The SHO then fired his rifle and the bullet went through his abdomen. Our neighbors tried to intervene and bring water for Shahbaz but the police fired shots in the air to scare them away. Shahbaz died on the spot in front of my eyes. There are many other witnesses t

In [6]:
retrieval_result.images

[[{'url': 'https://abcnews.go.com/US/state-suspends-brooklyn-funeral-homes-licence-stored-dozens/story?id=70451543',
   'imageUrl': 'https://s.abcnews.com/images/US/200430_wabc_funeralhomes_hpMain_4x5_992.jpg',
   'title': "State suspends Brooklyn funeral home's license after it stored dozens of bodies in rental trucks - ABC News",
   'thumbnailUrl': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQBwlh2cc2D9Nni1Te0y1QqCglJLpveaCNk_yz1KWPSaw5cnULX',
   'content': '[breakingAppeals court disqualifies Alina Habba from serving as US attorney for New Jersey](https://abcnews.go.com/US/appeals-court-disqualifies-alina-habba-serving-us-attorney/story?id=127999613)\n\n# State suspends Brooklyn funeral home\'s license after it stored dozens of bodies in rental trucks\n\nThe state health department and Brooklyn DA seek answers.\n\nBy[Aaron Katersky](https://abcnews.go.com/author/aaron_katersky), [Mark Crudele](https://abcnews.go.com/author/mark_crudele), and [Ivan Pereira](https://abcnews

In [8]:
from classification import DefaultClassifier, HuggingfaceClassifier, AverageEnsembleClassifier, LogRegEnsembleClassifier, RandomForestClassifier, NoTiebreakClassifier

# target = path + "data_store/vecstore/test/2k"
PIPELINE_NAME = "evtext"
classifier = NoTiebreakClassifier()  # DefaultClassifier()
if False:
    pipeline = MockPipeline(
        dumps=f"/mnt/data/factcheck/averitec-data/data_store/submissions/{split}_mmr+gpt4o-dfewshot-tiebrk-atype.pkl",
        classifier=NoTiebreakClassifier()
    )
else:
    pipeline = Pipeline(
        #dumps = "/mnt/data/factcheck/averitec-data/data_store/submissions/dev_mmr+gpt4o-dfewshot.pkl",
        #SubqueryRetriever(retriever),
        retriever,
        evidence_generator=DynamicFewShotBatchedEvidenceGenerator(), 
        classifier=classifier
    )

In [9]:
list(enumerate(retrieval_result.images))

[(0,
  [{'url': 'https://abcnews.go.com/US/state-suspends-brooklyn-funeral-homes-licence-stored-dozens/story?id=70451543',
    'imageUrl': 'https://s.abcnews.com/images/US/200430_wabc_funeralhomes_hpMain_4x5_992.jpg',
    'title': "State suspends Brooklyn funeral home's license after it stored dozens of bodies in rental trucks - ABC News",
    'thumbnailUrl': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQBwlh2cc2D9Nni1Te0y1QqCglJLpveaCNk_yz1KWPSaw5cnULX',
    'content': '[breakingAppeals court disqualifies Alina Habba from serving as US attorney for New Jersey](https://abcnews.go.com/US/appeals-court-disqualifies-alina-habba-serving-us-attorney/story?id=127999613)\n\n# State suspends Brooklyn funeral home\'s license after it stored dozens of bodies in rental trucks\n\nThe state health department and Brooklyn DA seek answers.\n\nBy[Aaron Katersky](https://abcnews.go.com/author/aaron_katersky), [Mark Crudele](https://abcnews.go.com/author/mark_crudele), and [Ivan Pereira](https

In [10]:
PIPELINE_NAME = "evtext"

In [11]:
split

'test'

In [12]:
submission = []
dump = []

for dp in tqdm(datapoints):
    pipeline_result = pipeline(dp)
    submission.append(pipeline_result.to_submission())
    dump.append(pipeline_result)
with open(f"{path}submissions/{split}_{PIPELINE_NAME}.json", "w") as f:
    json.dump(submission, f, indent=4)
with open(f"{path}submissions/{split}_{PIPELINE_NAME}.pkl", "wb") as f:
    pickle.dump(dump, f)

100%|██████████| 352/352 [00:28<00:00, 12.36it/s]


In [13]:
files = pipeline.evidence_generator.get_batch_files(path=f"/mnt/data/factcheck/averimatec/batch_jobs/{split}_{PIPELINE_NAME}", batch_size=352)

In [14]:
files

['/mnt/data/factcheck/averimatec/batch_jobs/test_evtext/batch_1.jsonl']

In [ ]:
batch_results = pipeline.evidence_generator.submit_and_await_batches(
    files, f"/mnt/data/factcheck/averimatec/batch_jobs/{split}_{PIPELINE_NAME}/output.jsonl"
)

  0%|          | 0/1 [00:00<?, ?it/s]

In [35]:
with open(f"/mnt/data/factcheck/averimatec/batch_jobs/{split}_{PIPELINE_NAME}/output.jsonl") as f:
    batch_results = [json.loads(line)["response"]["body"]["choices"][0]["message"]["content"] for line in f]

In [ ]:
new_result = pipeline.evidence_generator.update_pipeline_result(pipeline_result, batch_results, pipeline.classifier)

In [36]:
new_dump = []
pipeline.evidence_generator.fallback_gpt_generator.client.temperature = 0.5
for pipeline_result, batch_result in zip(dump[: len(batch_results)], batch_results):

    new_result = pipeline.evidence_generator.update_pipeline_result(
        pipeline_result, batch_result, pipeline.classifier
    )
    new_dump.append(new_result)

FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL


In [ ]:
new_result.to_submission()

In [37]:
new_dump = []
pipeline.evidence_generator.fallback_gpt_generator.client.temperature = .5
for pipeline_result, batch_result in zip(dump[:len(batch_results)], batch_results):
    new_result = pipeline.evidence_generator.update_pipeline_result(pipeline_result, batch_result, pipeline.classifier)
    new_dump.append(new_result)


In [39]:
new_result.to_submission()

{'id': 351,
 'claim': 'The national flag of India was placed at the Bangladesh University of Engineering and Technology campus gate in such a way that people could walk on it and disrespect it.',
 'questions': ['Does the viral image actually show the main gate of Bangladesh University of Engineering and Technology (BUET)?',
  'Does any source explicitly state that this image and the flag incident occurred at BUET?',
  'Does the same LinkedIn source also say that this incident was later found to be fake news by people associated with BUET?',
  'Do any of the provided sources independently confirm (beyond the LinkedIn caption) that students at BUET drew or placed an Indian national flag on the ground to be stepped on?',
  'Do any of the provided sources report on this image as part of a verified news story about disrespecting the Indian flag at BUET?',
  'According to the LinkedIn source, what is the status of the claim that students at BUET stepped on an Indian flag drawn on the ground?

In [43]:
with open(f"{path}submissions/{split}_{PIPELINE_NAME}2.json", "w") as f:
    json.dump([d.to_submission() for d in new_dump], f, indent=4)
with open(f"{path}submissions/{split}_{PIPELINE_NAME}2.pkl", "wb") as f:
    pickle.dump(new_dump, f)

In [41]:
f"{path}submissions/{split}_{PIPELINE_NAME}.json"

'/mnt/data/factcheck/averimatec/submissions/test_base64.json'

In [ ]:
(
    new_dump[1],
    new_dump[1].evidence_generation_result,
    new_dump[1].classification_result
)

In [ ]:
with open(f"/mnt/data/factcheck/averitec-data/data_store/submissions/{split}_{PIPELINE_NAME}.json", "w") as f:
    json.dump([d.to_submission() for d in new_dump], f, indent=4)
with open(f"/mnt/data/factcheck/averitec-data/data_store/submissions/{split}_{PIPELINE_NAME}.pkl", "wb") as f:
    pickle.dump(new_dump, f)

In [ ]:
with open(f"/mnt/data/factcheck/averitec-data/data_store/submissions/{split}_{PIPELINE_NAME}.json", "w") as f:
    json.dump([d.to_submission() for d in new_dump], f, indent=4)
with open(f"/mnt/data/factcheck/averitec-data/data_store/submissions/{split}_{PIPELINE_NAME}.pkl", "wb") as f:
    pickle.dump(new_dump, f)

## collapsible begin

In [ ]:
from IPython.display import display, Markdown, Latex

In [ ]:
knn_retrieval_result = retriever(datapoint)
display(Markdown("### 🗯️ " + datapoint.claim))
display(Markdown("*Retrieved by knn*\n\n"))
# sample 3
for r in knn_retrieval_result:
    newline = "\n"
    display(Markdown(f"**{r.metadata['url']}**\n\n{r.page_content[:256]}"))

In [ ]:
from retrieval import MmrFaissRetriever

mmr_retriever = MmrFaissRetriever(retriever.path)
mmr_retrieval_result = mmr_retriever(datapoint)
display(Markdown("### 🗯️ " + datapoint.claim))
display(Markdown("*Retrieved by MMR*\n\n"))
# sample 3
for r in mmr_retrieval_result:
    newline = "\n"
    display(Markdown(f"**{r.metadata['url']}**\n\n{r.page_content[:256]}"))

In [ ]:
subquery_retriever = SubqueryRetriever(retriever)
subquery_retrieval_result = subquery_retriever(datapoint)
display(Markdown("### 🗯️ " + datapoint.claim))
display(Markdown("*Retrieved by subqueries*\n\n"))
# sample 3
for r in subquery_retrieval_result:
    newline = "\n"
    display(Markdown(f"**{r.metadata['url']}**\n\n*{';'.join(r.metadata['queries'])}*\n\n{r.page_content[:256]}"))

In [ ]:
subquery_retrieval_result.metadata

## Collapsible section end

In [ ]:
evidence_generator = GptBatchedEvidenceGenerator("gpt-4o")
evidence_generation_result = evidence_generator(datapoint, retrieval_result)
evidence_generation_result

In [ ]:
evidence_generation_result.metadata["suggested_label"]

In [ ]:
datapoint.label

In [ ]:
classifier = DefaultClassifier()
classification_result = classifier(datapoint, evidence_generation_result, retrieval_result)
str(classification_result), classification_result

In [ ]:
datapoint2 = Datapoint.from_dict(dataset[16])
pipeline = Pipeline(retriever, evidence_generator, classifier)
pipeline_result = pipeline(datapoint2)
pipeline_result

In [ ]:
str(pipeline_result.classification_result), datapoint2.label

In [ ]:
pipeline_result.to_submission()

In [ ]:
# pickle dump pipeline result
import pickle
with open('data/pipeline_result.pkl', 'wb') as f:
    pickle.dump(pipeline_result, f)

In [ ]:
%run src/prediction/evaluate_veracity.py --label_file /mnt/data/factcheck/averitec-data/data/dev.json --prediction_file /mnt/data/factcheck/averitec-data/data_store/submission_dev_avg_clf.json

In [ ]:
import json, os
# crawl /mnt/data/factcheck/averitec-data/data_store/batch_jobs and each time you find gpt4o in folder name and "output" in filename, load the file and add its line count to line_counts
line_counts = []
for root, dirs, files in os.walk("/mnt/data/factcheck/averitec-data/data_store/batch_jobs"):
    #print(root, files)
    if "gpt4o" in root:
        for f in files:
            if "output" in f:
                print(os.path.join(root,f))
                with open(os.path.join(root,f)) as f:
                    line_counts.append(len(f.readlines()))
            

In [ ]:
line_counts

In [ ]:
()/sum(line_counts)